# Applied Machine Learning: Assignment \#09

### Sheetal Parikh
EN.605.631.81<br>
April 5, 2021
***
***

*Please refer to the Reinforcement Learning Jupyter notebook in course materials.* 

## Problem 1
*Describe the environment in the Nim learning model.*

The environment consists of the states, actions, and rewards of the system.  In the Nim learning model, the environment of the game can be visualized as a 3 x 11 board.  The 3 represents the three possible piles to choose from and the 11 represents $s_{t}$ + 1.  The variable, $s_{t}$ represents the the max number items per pile (which in our example is 10).  We are adding a 1 for if a pile has 0 items. The states are all of the possible choices from the board.  For example, one state would be a player removing 2 items from pile 3 (state = (3,2)).  Because the model has a finite number of states, the model also has a finite number of possible actions. The states and rewards are inputs of the Q-learning agent and the actions are the output. A further description of the rewards, states, and actions are stated in the other problems below.

***

## Problem 2

*Describe the agent in the Nim learning model.*

The goal of the agent in the Nim learning model is to determine the optimal method of removing items from the pile so that the last pile is cleared.  In this model, the agent can only remove items from the pile and can take anything from 1 item up to the amount of items left in the pile.  Also, the agent can only take items from the same pile in one turn.  The agent learns about the environment by updating the Q values and wants to identify the highest quality action in any given state. The Q-values estimate how much additional reward we can expect to accumulate through all the remaining steps in the episode if the agent is in a state and takes a certain action.  As the agent learns, it will always choose the action with the largest Q-value.  

***

## Problem 3

*Describe the reward and penalty in the Nim learning model.*

In the current Nim learning model, a reward of 100 is only given for the move that wins the game.  There isn't a penalty for losing the game.  Equation 3 from the module 9 notebook helps find the optimal policy for maximizing the expected value of the total reward.  The discount factor, gamma, prevents the total reward from going to infinity and since it is set at 0.80, it helps give higher weight to moves that give future rewards.  The alpha, or learning rate, determines how much of the new learned value would be used and is set at 1 which gives higher weight to recent rewards. In the model, the alpha is set to one since every winning state must be learned.

***

## Problem 4

*How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?*

To determine the number of possible states, we would find the product of the number of states per pile.  As stated in problem 1, each pile has a 11 states becauase we have 10 as the max items per pile plus 1 for if the the pile has 0 items remaining.  Therefore, in our Nim game, the possible states would be the following if we have 3 states:

(10 + 1) + (10 + 1) + (10 + 1) = 11 * 11 * 11 = $11^{3}$ = 1331 total possible states.



***

## Problem 5

*How many possible actions there could be in the Nim game with 10 items per pile and 3 piles total?*

In the Nim game, the agent can only remove items from the piles.  Therefore, to determine the possible actions in the Nim game with 10 items per pile and 3 piles, we would find the sum of the number of possible items per every pile.  For our example, the total possible actions would be:

10 + 10 + 10 = 30 total possible actions.

***

## Problem 6

*Find a way to improve the provided Nim game learning model. Do you think one can beat the Guru player? (Hint: How about penalizing the losses? Hint: It is indeed possible to find a better solution, which improves the way Q-learning updates its Q-table).*

In [1]:
#using code from Module 9 notebook

import numpy as np
from random import randint, choice

# The number of piles is 3

#number of games/episodes
n_episodes = 10000

# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
#using code from Module 9 notebook

def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    #
    return move, pile  # action

In [3]:
#using code from Module 9 notebook

Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}


def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        #
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s} {wins['A']:5d}  vs {b:>8s}{wins['B']:5d}      : "
                      f"{a:>8s} Win % = {round(100*(wins['A']/(wins['A'] + wins['B'])),1)}")
    #
    return wins['A'], wins['B']

In [4]:
#using code from Module 9 notebook

#qtable_loss and qtable_loss2 for the two new models
qtable, qtable_loss, qtable_loss2, Alpha, Gamma, Reward = None, None, None, 1.0, 0.80, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game
            #
            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2
            
# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)


In [5]:
#training model
nim_qlearn(100000)

In [6]:
#using code from Module 9 notebook

#improved Q-table update incorporating penalty amd using random exploration

def nim_qlearn_wloss(_n):
    global qtable_loss
    # based on max items per pile
    qtable_loss = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        #for the losing move
        st0 = None
        earlier_move = None
        earlier_pile = None
        while True:  # while game not finished
            # make a random move - exploration
            #move, pile = nim_guru(st1)
            #move, pile = explore(st1)
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update2(Reward, st1, move, pile, 0)  # winning
                #qtable_update2(-100, st0, earlier_move, earlier_pile, 0)  # losing
                #reward based on difference in next state value and the current state’s value
                qtable_update2((Alpha * (Reward + Gamma * np.max(qtable_loss[st1]) - np.max(qtable_loss[st0]))), 
                               st0, earlier_move, earlier_pile, 0)  # losing
                break  # new game
            #
            qtable_update2(0, st1, move, pile, np.max(qtable_loss[st2[0], st2[1], st2[2]]))
            earlier_move = move
            earlier_pile = pile
            st0 = st1
            st1 = st2

# Equation 3 - update the qtable
def qtable_update2(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable_loss[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)


In [7]:
#using code from Module 9 notebook

def nim_qlearner_loss(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable_loss[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    #
    return move, pile  # action

In [8]:
#using code from Module 9 notebook

#improved Q-table update incorporating penalty and using nim-guru exploration

def nim_qlearn_wloss2(_n):
    global qtable_loss2
    # based on max items per pile
    qtable_loss2 = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        #for the losing move
        st0 = None
        earlier_move = None
        earlier_pile = None
        while True:  # while game not finished
            # make a move - nim guru exploration
            move, pile = nim_guru(st1)
            #move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update3(Reward, st1, move, pile, 0)  # woinning
                #qtable_update3(-100, st0, earlier_move, earlier_pile, 0)  # losing
                #reward based on difference in next state value and the current state’s value
                qtable_update3((Alpha * (Reward + Gamma * np.max(qtable_loss2[st1]) - np.max(qtable_loss2[st0]))), 
                               st0, earlier_move, earlier_pile, 0)  # losing
                break  # new game
            #
            qtable_update3(0, st1, move, pile, np.max(qtable_loss2[st2[0], st2[1], st2[2]]))
            earlier_move = move
            earlier_pile = pile
            st0 = st1
            st1 = st2

# Equation 3 - update the qtable
def qtable_update3(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable_loss2[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [9]:
#using code from Module 9 notebook

def nim_qlearner_loss2(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable_loss2[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    #
    return move, pile  # action

In [10]:
%%time

#adding new models to Engines and training 

#adding to Engines
Engines['Qlearner_wloss'] = nim_qlearner_loss
Engines['Qlearner_wloss2'] = nim_qlearner_loss2

#training
nim_qlearn_wloss(100000)
nim_qlearn_wloss2(100000)

Wall time: 57 s


In [11]:
# Play games when Guru goes first
#n_episodes = 10000

play_games(n_episodes, 'Guru', 'Guru')
play_games(n_episodes, 'Guru', 'Random') ;
play_games(n_episodes, 'Guru', 'Qlearner') ;
play_games(n_episodes, 'Guru', 'Qlearner_wloss')
play_games(n_episodes, 'Guru', 'Qlearner_wloss2')

10000 games,     Guru  9371  vs     Guru  629      :     Guru Win % = 93.7
10000 games,     Guru  9980  vs   Random   20      :     Guru Win % = 99.8
10000 games,     Guru  9983  vs Qlearner   17      :     Guru Win % = 99.8
10000 games,     Guru  9979  vs Qlearner_wloss   21      :     Guru Win % = 99.8
10000 games,     Guru  9389  vs Qlearner_wloss2  611      :     Guru Win % = 93.9


(9389, 611)

In [12]:
#play games when Random goes first

play_games(n_episodes, 'Random', 'Random')
play_games(n_episodes, 'Random', 'Guru') ;
play_games(n_episodes, 'Random', 'Qlearner') ;
play_games(n_episodes, 'Random', 'Qlearner_wloss')
play_games(n_episodes, 'Random', 'Qlearner_wloss2')

10000 games,   Random  5001  vs   Random 4999      :   Random Win % = 50.0
10000 games,   Random    78  vs     Guru 9922      :   Random Win % = 0.8
10000 games,   Random  2972  vs Qlearner 7028      :   Random Win % = 29.7
10000 games,   Random  2955  vs Qlearner_wloss 7045      :   Random Win % = 29.5
10000 games,   Random    99  vs Qlearner_wloss2 9901      :   Random Win % = 1.0


(99, 9901)

In [13]:
#play games when Qlearner goes first

play_games(n_episodes, 'Qlearner', 'Qlearner')
play_games(n_episodes, 'Qlearner', 'Guru') ;
play_games(n_episodes, 'Qlearner', 'Random') ;
play_games(n_episodes, 'Qlearner', 'Qlearner_wloss')
play_games(n_episodes, 'Qlearner', 'Qlearner_wloss2')

10000 games, Qlearner 10000  vs Qlearner    0      : Qlearner Win % = 100.0
10000 games, Qlearner   316  vs     Guru 9684      : Qlearner Win % = 3.2
10000 games, Qlearner  7211  vs   Random 2789      : Qlearner Win % = 72.1
10000 games, Qlearner 10000  vs Qlearner_wloss    0      : Qlearner Win % = 100.0
10000 games, Qlearner  1017  vs Qlearner_wloss2 8983      : Qlearner Win % = 10.2


(1017, 8983)

In [14]:
#play games when improved Qlearner_wLoss goes first

play_games(n_episodes, 'Qlearner_wloss', 'Qlearner_wloss')
play_games(n_episodes, 'Qlearner_wloss', 'Random') ;
play_games(n_episodes, 'Qlearner_wloss', 'Qlearner') ;
play_games(n_episodes, 'Qlearner_wloss', 'Guru')
play_games(n_episodes, 'Qlearner_wloss', 'Qlearner_wloss2')

10000 games, Qlearner_wloss 10000  vs Qlearner_wloss    0      : Qlearner_wloss Win % = 100.0
10000 games, Qlearner_wloss  7234  vs   Random 2766      : Qlearner_wloss Win % = 72.3
10000 games, Qlearner_wloss 10000  vs Qlearner    0      : Qlearner_wloss Win % = 100.0
10000 games, Qlearner_wloss   312  vs     Guru 9688      : Qlearner_wloss Win % = 3.1
10000 games, Qlearner_wloss   987  vs Qlearner_wloss2 9013      : Qlearner_wloss Win % = 9.9


(987, 9013)

In [15]:
play_games(n_episodes, 'Qlearner_wloss2', 'Qlearner_wloss2')
play_games(n_episodes, 'Qlearner_wloss2', 'Random') ;
play_games(n_episodes, 'Qlearner_wloss2', 'Qlearner') ;
play_games(n_episodes, 'Qlearner_wloss2', 'Guru')
play_games(n_episodes, 'Qlearner_wloss2', 'Qlearner_wloss')

10000 games, Qlearner_wloss2  9395  vs Qlearner_wloss2  605      : Qlearner_wloss2 Win % = 94.0
10000 games, Qlearner_wloss2  9972  vs   Random   28      : Qlearner_wloss2 Win % = 99.7
10000 games, Qlearner_wloss2 10000  vs Qlearner    0      : Qlearner_wloss2 Win % = 100.0
10000 games, Qlearner_wloss2  9408  vs     Guru  592      : Qlearner_wloss2 Win % = 94.1
10000 games, Qlearner_wloss2 10000  vs Qlearner_wloss    0      : Qlearner_wloss2 Win % = 100.0


(10000, 0)

I tried two Nim learning models. For both models, the winning move continues to receive a reward of 100.  However the losing move receives a reward that is applied to the difference between the discounted value of the move in the next state and the current state’s value.  Therefore, "better" bad moves will have higher rewards. The gamma discount rate is also applied to make sure no moves are heaviliy weighed.  When I tried adding a penalty of a set value, for example, -100, the model performance worsened.  The first model, uses the random method of exploration.  This model doesn't seem to have any improvement to the original nim learner model. I wasn't able to figure out how to improve the model further.  Therefore, the second model, used the nim guru engine to explore the possible states. This model had an above 90% winning rate against the Guru player but only because it used the nim-guru engine.

***
## References

https://towardsdatascience.com/getting-started-with-reinforcement-q-learning-77499b1766b6

https://medium.com/@edkins.sarah/intro-to-how-q-table-agents-learn-26c163e42db7

https://www.duo.uio.no/bitstream/handle/10852/73236/Solving-Nim-by-the-Use-of-Machine-Learning.pdf?sequence=1&isAllowed=y

https://towardsdatascience.com/q-learning-54b841f3f9e4
    
